In [15]:
def plot_time_dependent_state(gdf)  :

    # folder
    # ===================================================
    if not os.path.exists('data/FigurePlot/plot_time_dependent_state' ):
        os.makedirs( 'data/FigurePlot/plot_time_dependent_state' )
        print('create directory : plot_time_dependent_state' )
        
    folder_path = 'data/FigurePlot/plot_time_dependent_state/'

    
    # function
    # ===================================================
    def state_map(x):
        if x == "LOS" : 
            state = 2
            return state
        if x == 'NLOS': 
            state = 1
            return state
        state = 0
        return state
    
    def color_map(x):
        if x == "LOS" : 
            color = '#55A868'
            return color
        if x == 'NLOS': 
            color = '#C44E52'
            return color
        color = '#4C72B0'
        return color
    
    # plot
    # ===================================================
    buffer_df = gdf
    
    location_list = buffer_df["location"].unique()
    for location in location_list :
    
        gdf = buffer_df[buffer_df['location']==location]
    
    
        SAT_list = gdf["SAT_name"].unique()
        for sat in SAT_list : 

            sat_gdf = gdf[gdf['SAT_name']==sat]

            sat_gdf["state"] = sat_gdf["State"].map(state_map)
            sat_gdf["color"] = sat_gdf["State"].map(color_map)
            #sat_gdf['rise_set'] = np.nan
            sat_gdf['rise_set'] = np.nan

            sat_gdf = sat_gdf.reset_index(drop=True)
            OFR_index_list =  sat_gdf[sat_gdf['State']=='out_of_range'].index.values.tolist()

            list_set = []
            list_rise = []

            for index in OFR_index_list :
                try :
                    if sat_gdf.iloc[index+1,:]['State'] != 'out_of_range' :
                        #print('rise :',index+1)
                        sat_gdf['rise_set'][index+1]='rise'
                        #print('rise ',index)
                        list_rise.append(index+1)
                except :
                    #print('error : rise :',str(index))
                    pass
                try :
                    if sat_gdf.iloc[index-1,:]['State'] != 'out_of_range' :
                        #print('set :',index-1)
                        sat_gdf['rise_set'][index-1]='set'
                        list_set.append(index-1)
                except :
                    #print('error : set  :',str(index))
                    pass


            #print('satellite:',str(sat))
            #print(list_set)
            #print(list_rise)

            np_set  = np.array(list_set)
            np_rise = np.array(list_rise)

            np_list = np.asarray([list_rise,list_set])
            #tuple_xlim = tuple(map(tuple, np_list.T.tolist()))
            #list_xlim = np_list.T.tolist()
            list_xlim = np_list.T.tolist()

            #print(list_xlim)

            np_interval = np_set - np_rise
            list_interval = np_interval.tolist()
            #print('set/rise',str(len(list_set)),'/',str(len(list_rise)))


            fig_num = len(list_set)

            #plt.figure(figsize=(15,15))
            #plt.title('SAT:' + sat + ',name:' + name)
            fig, ax = plt.subplots(1,fig_num,
                                   sharex=True,
                                   sharey=True,
                                   figsize=(30,15),
                                   gridspec_kw={'width_ratios': list_interval})
            fig.subplots_adjust(bottom=0.22)
            plt.xticks([])
            plt.yticks([])

            #plt.tight_layout()
            #plt.title('Satellite : '+str(sat))
            #plt.suptitle('Satellite : '+str(sat)+',location : '+str(location), fontsize = 24)
            fig.text(0.5, 0.04, 'Time', ha='center', va='center', fontsize = 36)
            fig.text(0.06, 0.5, 'State', ha='center', va='center', rotation='vertical', fontsize = 36)

            for i in range(0,len(list_set)):

                #ax[i] = plt.subplot(1,fig_num,i+1)
                ax[i] = fig.add_subplot(1,fig_num,i+1)


                plt.xticks(rotation=90)

                if i != 0 :
                    #print (i)
                    #ax[i].get_yaxis().get_offset_text().set_visible(False)
                    #ax[i].set_yticks([])
                    plt.setp( ax[i].get_yticklabels(), visible=False)

                min_xlim = sat_gdf[sat_gdf.index==list_xlim[i][0]]['TIME']
                max_xlim = sat_gdf[sat_gdf.index==list_xlim[i][1]]['TIME']
                #print('##')
                #print(min_xlim,' ',max_xlim)


                ax[i].set_xlim(min_xlim,max_xlim)

                #print('##')
                #print(min_xlim,' ',max_xlim)



                ax[i].set_ylim([0.5,2.5])
                ax[i].set_yticks((1,2)) 
                ax[i].set_yticklabels(("NLOS", "LOS"),fontsize=30)

                #ax[i].set_ylim(0,90)
                #plt.xlim((min_xlim,max_xlim)) 

                ax[i].spines['right'].set_visible(False)
                ax[i].spines['left'].set_visible(False)
                ax[i].plot_date(sat_gdf['TIME'], sat_gdf['state'], 
                                '--', markersize=10,label = str(sat),
                                c = '#4C72B0')

                fliter_LOS  = (sat_gdf['State']=='LOS')
                fliter_NLOS = (sat_gdf['State']=='NLOS') 

                sat_gdf_LOS = sat_gdf[fliter_LOS]
                ax[i].plot_date(sat_gdf_LOS['TIME'], sat_gdf_LOS['state'], 
                                '.', markersize=15,label = str(sat),
                                c = '#55A868')
                sat_gdf_NLOS = sat_gdf[fliter_NLOS]
                ax[i].plot_date(sat_gdf_NLOS['TIME'], sat_gdf_NLOS['state'], 
                                '.', markersize=15,label = str(sat),
                                c = '#C44E52')
                # 斷軸
                # ========================================================================
                y_d = .03
                x_d = 1/list_interval[i]
                # how big to make the diagonal lines in axes coordinates
                # arguments to pass to plot, just so we don't keep repeating them
                kwargs = dict(transform=ax[i].transAxes, color='k', clip_on=False)
                if i != len(list_set)-1 :
                    ax[i].plot((1 - x_d, 1 + x_d), (1 - y_d, 1 + y_d), **kwargs)  # bottom-right diagonal
                    ax[i].plot((1 - x_d, 1 + x_d), (-y_d, +y_d), **kwargs)        # top-right diagonal
                if i != 0 :
                    ax[i].plot((-x_d, +x_d), (1 - y_d, 1 + y_d), **kwargs)        # bottom-left diagonal
                    ax[i].plot((-x_d, +x_d), (-y_d, +y_d), **kwargs)              # top-left diagonal
                # ========================================================================


                #print(list_xlim[i])
                #ax.set_xticks(rotation=90)

                #timeFmt = mdates.DateFormatter('%y:%m:%d %H:%M:%S')
                timeFmt = mdates.DateFormatter('%b %d, %H:%M')
                ax[i].xaxis.set_major_locator(mdates.MinuteLocator())
                ax[i].xaxis.set_major_formatter(timeFmt)

                my_xticks = ax[i].get_xticks()
                ax[i].set_xticks([my_xticks[0], my_xticks[-1]],rotation=90)
                ax[i].tick_params(axis='both', which='major', labelsize=24)
            #plt.legend()
            plt.savefig(folder_path + str(sat) + '_' + str(location) +'.png')
            #plt.show() 
            #print(list_interval)
            plt.close() 